In [22]:
# IMPORTING REQUIRED MODULES

import os
import random
import numpy as np

from keras.utils import np_utils
from tensorflow.keras.preprocessing import image

from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model


In [23]:
# PREPARING DATA

data_path = '/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train'
pokemons = os.listdir(data_path)
print(pokemons)

image_data = []
labels = []
count = 0

for ix in pokemons:
    path = os.path.join(data_path, ix)
    print(path, count) 
    for im in os.listdir(path):
        try:
            img = image.load_img(os.path.join(path, im), target_size = (224,224))
            img_array = image.img_to_array(img)
            image_data.append(img_array)
            labels.append(count)
        except Exception as e:
            print(os.path.join(path, im))
            print(e)
    count += 1

combined_dataset = list(zip(image_data, labels))
random.shuffle(combined_dataset)
image_data[:], labels[:] = zip(*combined_dataset)

X_train = np.array(image_data)

Y_train = np.array(labels)
Y_train = np_utils.to_categorical(Y_train)


['Charmander', 'Fearow', 'Psyduck', 'Dratini', 'Squirtle', 'Meowth', 'Pikachu', 'Aerodactyl', 'Bulbasaur', 'Spearow']
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Charmander 0


/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Fearow 1
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Fearow/8f2d7986ab76482fb76cac6a91b66de4.svg
cannot identify image file <_io.BytesIO object at 0x7f574a74c310>
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Fearow/2c8d5221e92444f2b9477350e405bc54.svg
cannot identify image file <_io.BytesIO object at 0x7f574a67f810>
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Psyduck 2
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Dratini 3
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Squirtle 4
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Meowth 5
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Pikachu 6
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Aerodactyl 7
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Bulbasaur 8
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifie

In [24]:
# LOADING THE BASE RESNET50 FROM KERAS

model = ResNet50(include_top = False, weights = 'imagenet', input_shape = (224,224,3))
print(model.summary())


Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [25]:
# CREATING OUR OWN CLASSIFIER

av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256, activation = 'relu')(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(10, activation = 'softmax')(d1)


In [26]:
# CONNECTING PRE-TRAINED RESNET50 WITH OUR CLASSIFIER
# FEATURE EXTRACTOR STRATEGY

model_new = Model(inputs = model.input, outputs = fc2)
model_new.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [27]:
# PREDICTING WITHOUT TRAINING OUR CLASSIFIER

image_path = '/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Pikachu/00000007.jpg'
img = image.load_img(image_path, target_size = (224,224))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
# x = preprocess_input(x)

pred = model_new.predict(x)
print(pokemons[np.argmax(pred)])

1/1 [==============================] - 1s 871ms/step
Aerodactyl


In [28]:
# COMPILING MODEL FOR TRAINING

adam = Adam(lr = 0.00003)
model_new.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

In [29]:
# CHECKING ALL LAYERS OF THE MODEL

for ix in range(len(model_new.layers)):
    print(ix, model_new.layers[ix])

0 <keras.engine.input_layer.InputLayer object at 0x7f580d9ed5b0>
1 <keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x7f574a5e0130>
2 <keras.layers.convolutional.conv2d.Conv2D object at 0x7f573019fee0>
3 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f57d0013e80>
4 <keras.layers.core.activation.Activation object at 0x7f5766318760>
5 <keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x7f5756fefbe0>
6 <keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7f574a82d9a0>
7 <keras.layers.convolutional.conv2d.Conv2D object at 0x7f574a5e05b0>
8 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f574a7550d0>
9 <keras.layers.core.activation.Activation object at 0x7f574a5e0d00>
10 <keras.layers.convolutional.conv2d.Conv2D object at 0x7f580d9f81f0>
11 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f580d9f8d60>
12 <keras.layers.core.activation.Activation object at 0x7f574

In [30]:
# FREEZING ALL LAYERS EXCEPT A FEW FOR TRAINING ON DATASET

for ix in range(175):
    model_new.layers[ix].trainable = False
print(model_new.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [31]:
# TRAINING ONLY TRAINABLE LAYERS OF MODEL

hist = model_new.fit(X_train, Y_train, shuffle = True, batch_size = 16, epochs = 20, validation_split = 0.20)

Epoch 1/20
83/83 [==============================] - 49s 198ms/step - loss: 1.6327 - accuracy: 0.5365 - val_loss: 3417.4919 - val_accuracy: 0.1520
Epoch 2/20
83/83 [==============================] - 15s 186ms/step - loss: 1.0858 - accuracy: 0.6565 - val_loss: 14.2879 - val_accuracy: 0.3678
Epoch 3/20
83/83 [==============================] - 15s 186ms/step - loss: 0.7602 - accuracy: 0.7789 - val_loss: 7.7290 - val_accuracy: 0.2158
Epoch 4/20
83/83 [==============================] - 16s 187ms/step - loss: 0.6302 - accuracy: 0.8245 - val_loss: 1.3982 - val_accuracy: 0.6869
Epoch 5/20
83/83 [==============================] - 15s 186ms/step - loss: 0.4869 - accuracy: 0.8556 - val_loss: 1.6986 - val_accuracy: 0.5653
Epoch 6/20
83/83 [==============================] - 15s 183ms/step - loss: 0.3510 - accuracy: 0.8936 - val_loss: 1.1730 - val_accuracy: 0.7477
Epoch 7/20
83/83 [==============================] - 15s 184ms/step - loss: 0.3675 - accuracy: 0.8974 - val_loss: 3.3960 - val_accuracy: 0.

In [46]:
# PREDICTING MODEL AFTER TRAINING LAST LAYERS WITH OUR DATA

image_path = '/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/8f2d7986ab76482fb76cac6a91b66de4.png'
img = image.load_img(image_path, target_size = (224,224))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
# x = preprocess_input(x)

pred = model_new.predict(x)
print(pokemons[np.argmax(pred)])

1/1 [==============================] - 0s 26ms/step
Fearow
